In [ ]:
#use virtual environment to run this code
!pip install pymc3

In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm

In [ ]:
# Read individual stock market datasets and combine them into one DataFrame
files = ['WOR.csv', 'WOW.csv', 'WPL.csv', 'WTC.csv']

In [ ]:
data = pd.DataFrame()

for file in files:
    stock_data = pd.read_csv(file, index_col='Date', parse_dates=True)
    stock_data = stock_data[['Adj Close']]  # Keep only the 'Adjusted Close' column
    stock_data.columns = [file.split('.')[0]]  # Rename the column to the company's name
    if data.empty:
        data = stock_data
    else:
        data = data.join(stock_data, how='outer')

In [ ]:
# Calculate daily returns and drop missing values
returns = data.pct_change().dropna()

# Calculate the covariance matrix of the daily returns
cov_matrix = returns.cov()

# Define Bayesian regression model for portfolio optimization
n_assets = len(data.columns)

In [ ]:
with pm.Model() as portfolio_model:
    # Define prior distributions for the asset weights
    weights = pm.Dirichlet('weights', a=np.ones(n_assets), shape=n_assets)

    # Calculate the portfolio returns using the weights
    portfolio_return = pm.Deterministic('portfolio_return', pm.math.dot(returns.mean(), weights))

    # Calculate the portfolio volatility using the weights and covariance matrix
    portfolio_volatility = pm.Deterministic('portfolio_volatility',
                                            pm.math.sqrt(pm.math.dot(pm.math.dot(weights, cov_matrix), weights.T)))

    # Define the utility function (e.g., Sharpe ratio) to optimize
    sharpe_ratio = pm.Deterministic('sharpe_ratio', portfolio_return / portfolio_volatility)

    # Maximize the utility function
    pm.Potential('sharpe_ratio', sharpe_ratio)

In [ ]:
# Perform Bayesian optimization to find the optimal portfolio weights
with portfolio_model:
    trace = pm.sample(draws=2000, tune=1000, target_accept=0.95, return_inferencedata=False)

In [ ]:
# Display the results
for file, weight in zip(files, optimal_weights):
    print(f"{file.split('.')[0]}: {weight:.4f}")

print("\nOptimal Portfolio Weights: ", optimal_weights)